In [ ]:
from IPython.display import display, clear_output
clear_output(wait=True)

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import pandas as pd
from pymongo import MongoClient
from matplotlib.animation import FuncAnimation

# === MongoDB Setup ===
hostip = "192.168.0.21" 
client = MongoClient(f"mongodb://{hostip}:27017")
violation_col = client["awas_db"]["violation"]
latest_time = "0000-00-00T00:00:00"  # lowest ISO string

# === Plot Setup ===
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 9))
# fig.tight_layout(pad=4)

# === Annotation Helpers ===
def annotate_max(x, y, ax):
    if not y: return
    ymax = max(y)
    xpos = y.index(ymax)
    xmax = x[xpos]
    text = 'Max: Time={}, Value={:.2f}'.format(xmax, ymax)
    ax.annotate(text, xy=(xmax, ymax), xytext=(xmax, ymax + 5),
                arrowprops=dict(facecolor='red', shrink=0.05),)

def annotate_min(x, y, ax):
    if not y: return
    ymin = min(y)
    xpos = y.index(ymin)
    xmin = x[xpos]
    text = 'Min: Time={}, Value={:.2f}'.format(xmin, ymin)
    ax.annotate(text, xy=(xmin, ymin), xytext=(xmin, ymin + 5),
                arrowprops=dict(facecolor='orange', shrink=0.05),)

def annotate_avg(x, y, ax):
    if not y: return
    avg_val = sum(y) / len(y)
    ax.axhline(avg_val, color='blue', linestyle='--', linewidth=1)
    text = 'Avg: {:.2f}'.format(avg_val)
    ax.annotate(text, xy=(x[-1], avg_val), xytext=(x[-1], avg_val + 5),
                arrowprops=dict(facecolor='blue', arrowstyle='->'), fontsize=9)

# === MongoDB Fetch Helper ===
def fetch_latest_data(since_time):
    pipeline = [
        {"$unwind": "$violations"},
        {"$project": {
            "timestamp": "$violations.timestamp_start",
            "recorded_speed": "$violations.recorded_speed"
        }},
        {"$match": { "timestamp": { "$gt": since_time }}}
    ]
    return pd.DataFrame(list(violation_col.aggregate(pipeline)))

# === Update Function ===
def update(frame):
    global latest_time
    df = fetch_latest_data(latest_time)

    if df.empty:
        print("[Tick] No new data.")
        return

    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df.dropna(subset=["timestamp"], inplace=True)
    df["rounded_time"] = df["timestamp"].dt.floor("min")
    latest_time = df["timestamp"].max().isoformat()

    vdf = df.groupby("rounded_time").size().reset_index(name="num_violations")
    sdf = df.groupby("rounded_time")["recorded_speed"].mean().reset_index(name="avg_speed")

    # Clear previous plots
    ax1.cla()
    ax2.cla()

    # === Violations Plot ===
    x1 = list(vdf["rounded_time"])
    y1 = list(vdf["num_violations"])
    ax1.plot(x1, y1, 'b-o')
    ax1.set_title("Number of Violations vs Time")
    ax1.set_ylabel("Violations")
    ax1.set_xlabel("Time")
    annotate_max(x1, y1, ax1)
    annotate_min(x1, y1, ax1)
    #     annotate_avg(x1, y1, ax1)

    # === Speed Plot ===
    x2 = list(sdf["rounded_time"])
    y2 = list(sdf["avg_speed"])
    ax2.plot(x2, y2, 'g-o')
    ax2.set_title("Average Speed vs Time")
    ax2.set_ylabel("Speed (km/h)")
    ax2.set_xlabel("Time")
    annotate_max(x2, y2, ax2)
    annotate_min(x2, y2, ax2)
    annotate_avg(x2, y2, ax2)

    fig.autofmt_xdate()
    fig.canvas.draw_idle()
    print(f"[Tick] Plotted {len(df)} new records. Latest: {latest_time}")

# === Animate ===
ani = FuncAnimation(fig, update, interval=2000)
plt.show()
